**第3章 RFRスワップとマルチカーブ**

In [1]:
from myABBR import * ; import myUTIL as mu
crvDATA = [('depo','1d',-0.009),  ('swap','1w',-0.01261),('swap','2w',-0.01469),
           ('swap','1m',-0.01807),('swap','3m',-0.01919),('swap','6m',-0.01043),
           ('swap','9m',0.00022), ('swap','12m',0.0125), ('swap','18m',0.03125),
           ('swap','2y',0.04875), ('swap','3y',0.07375), ('swap','4y',0.09479), 
           ('swap','5y',0.11854), ('swap','7y',0.19146)                       ]

In [2]:
def makeTonaCurve(crvDATA):
  
  # 1.指標金利オブジェクト
  tnCrvHDL = ql.RelinkableYieldTermStructureHandle()    
  tonaIX = ql.OvernightIndex('TONA', Tp0, jpyFX, calJP, dcA365, tnCrvHDL)
  # 2. カーブヘルパー
  cHelper, tnParRT = [], []
  for knd, tnr, rt in crvDATA:
      if knd == 'depo':
          cHelper.append(ql.DepositRateHelper(mu.sqHDL(rt/100),tonaIX)) 
      if knd == 'swap':
          cHelper.append(ql.OISRateHelper(Tp2, pD(tnr), mu.sqHDL(rt/100),tonaIX))
      tnParRT.append(rt/100)                                # パーレート用リスト
  # カーブオブジェクト
  tnCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp0, calJP, cHelper, dcA365)
  tnCrvHDL.linkTo(tnCrvOBJ) ; tnCrvOBJ.enableExtrapolation()
  return [tonaIX, tnCrvOBJ, tnCrvHDL, tnParRT]        # 4つのオブジェクトを戻す

In [3]:
tradeDT = jDT(2022,8,19) ; setEvDT(tradeDT)
tonaIX, tnCrvOBJ, tnCrvHDL, tnParRT = makeTonaCurve(crvDATA)
dfTONA = pd.DataFrame([(dt,df,pr,tnCrvOBJ.zeroRate(dt,dcA365,cmpdSPL,freqA).rate()) 
                        for (dt,df),pr in zip(tnCrvOBJ.nodes()[1:], tnParRT)  ],
                        columns=['date','tonaDF','parRT','zeroRT'])
print("決済日(reference): ", tnCrvOBJ.referenceDate().ISO())
fmtSCF.update(tonaDF='{:.10f}')
display(dfTONA[:4].style.format(fmtSCF))

決済日(reference):  2022-08-19


,date,tonaDF,parRT,zeroRT
0,"August 22nd, 2022",1.0000007397,-0.009000%,-0.009000%
1,"August 30th, 2022",1.0000035036,-0.012610%,-0.011625%
2,"September 6th, 2022",1.0000067198,-0.014690%,-0.013626%
3,"September 26th, 2022",1.0000179178,-0.018070%,-0.017210%


In [4]:
tnCrvOBJ.discount(jDT(2022,8,23))

1.0000010852067613

In [5]:
# TONAカーブの5%シフト
tnSFtOBJ= ql.ZeroSpreadedTermStructure(
                                tnCrvHDL, mu.sqHDL(5.0/100), cmpdSPL, freqA, dcA365)
tnSFtHDL= ql.YieldTermStructureHandle(tnSFtOBJ)
tnSFtIX = ql.OvernightIndex('TONA', Tp0, jpyFX, calJP, dcA365, tnSFtHDL)
dfSFT   = pd.DataFrame([(tnSFtOBJ.zeroRate(dt, dcA365, cmpdSPL, freqA).rate(),
            tnSFtOBJ.discount(dt) ) for dt in dfTONA.date], columns=['shftRT','DF'])
dfTnSFT = pd.concat([dfTONA,dfSFT], axis=1)
display(dfTnSFT[:4].style.format(fmtSCF))
# シフトカーブの使用
tnCrvOBJ = tnSFtOBJ; tnCrvHDL = tnSFtHDL; tonaIX = tnSFtIX 

,date,tonaDF,parRT,zeroRT,shftRT,DF
0,"August 22nd, 2022",1.0000007397,-0.009000%,-0.009000%,4.991000%,0.99958995
1,"August 30th, 2022",1.0000035036,-0.012610%,-0.011625%,4.988375%,0.99849891
2,"September 6th, 2022",1.0000067198,-0.014690%,-0.013626%,4.986374%,0.99754700
3,"September 26th, 2022",1.0000179178,-0.018070%,-0.017210%,4.982790%,0.99483921


In [6]:
# 3.スワップ条件  payRcv:ql.OvernightIndexedSwap.Payer = 1
effDT,               matDT,    payRcv, payLag, ntlAMT,     cpnRT,  sprdRT  =\
jDT(2022,8,23), jDT(2022,8,30),  1,      2,   10_000_000, 5.0/100,   0.0

# 4.スケジュール及びスワップオブジェクトの作成、エンジン設定
fixSCD  = ql.Schedule(effDT, matDT, pdFreqA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
swapOBJ = ql.OvernightIndexedSwap(
                 payRcv, ntlAMT, fixSCD, cpnRT, dcA365, tonaIX, sprdRT, payLag)
swapOBJ.setPricingEngine(ql.DiscountingSwapEngine(tnCrvHDL))

# 5. 計算結果
pd.DataFrame([  
    ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
    ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
    ['フェアスプレッド',swapOBJ.fairSpread()] ], columns=['計算結果', ''])\
                                                 .style.format({'':'{:.7f}'})    

,計算結果,
0,固定レグ時価,-9572.0373340
1,変動レグ時価,9542.6781729
2,スワップ時価 NPV,-29.3591611
3,フェアレート,0.0498466
4,フェアスプレッド,0.0001534


In [7]:
# 固定レグ:0
fmtSCF['amount']='{:,.4f}'                           # fmtSCFのamount桁数 修正
dfFixCF = mu.swapCashFlow(swapOBJ, tnCrvOBJ, 0)
dfFixCF.style.format(fmtSCF)

,nominal,accStart,accEnd,payDate,days,rate,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan,0.99945344
1,"10,000,000.00",2022-08-23,2022-08-30,2022-09-01,7,5.000000%,"9,589.0411",0.99822675


In [8]:
# 変動レグ:1
dfFltCF = mu.swapCashFlow(swapOBJ, tnCrvOBJ, 1)
dfFltCF.style.format(fmtSCF)

,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan%,nan,0.99945344
1,2022-08-29,2022-08-23,2022-08-30,2022-09-01,7,4.984664%,0.000%,"9,559.6298",0.99822675


In [9]:
# フェアレート
effDF  = tnCrvOBJ.discount(effDT)
matDF  = tnCrvOBJ.discount(matDT)
annFCT = mu.calcAnnuity(fixSCD, tnCrvOBJ)
swpRT  = (effDF - matDF) / annFCT
print( f'effDF:{effDF:.8f}',   f' matDF:{matDF:.8f}',
    f' annuity:{annFCT:.8f}', f' swapRT:{swpRT:.6%}')

effDF:0.99945344  matDF:0.99849891  annuity:0.01914929  swapRT:4.984664%


In [10]:
# Schedule 3番目引数をpdFreqD, payLagは0へ
fixSCDdy  = ql.Schedule(effDT, matDT, pdFreqD, calJP, mFLLW, mFLLW, dtGENb, EoMf )
swapOBJdy = ql.OvernightIndexedSwap(payRcv, ntlAMT, fixSCDdy, cpnRT, dcA365, tonaIX)
dfCFdy    = mu.swapCashFlow(swapOBJdy, tnCrvOBJ, 1)  # 変動レグ:1
dfCFdy.style.format(fmtSCF)

,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan%,nan,0.99945344
1,2022-08-23,2022-08-23,2022-08-24,2022-08-24,1,4.984664%,0.000%,"1,365.6614",0.99931697
2,2022-08-24,2022-08-24,2022-08-25,2022-08-25,1,4.983983%,0.000%,"1,365.4749",0.99918053
3,2022-08-25,2022-08-25,2022-08-26,2022-08-26,1,4.983303%,0.000%,"1,365.2885",0.99904413
4,2022-08-26,2022-08-26,2022-08-29,2022-08-29,3,4.982623%,0.000%,"4,095.3064",0.99863516
5,2022-08-29,2022-08-29,2022-08-30,2022-08-30,1,4.980583%,0.000%,"1,364.5433",0.99849891


In [11]:
# 8月29日のrate
aug29Fwd = (dfCFdy.DF[4]/dfCFdy.DF[5] -1)*365
print(f'TONAレート(Aug29): {aug29Fwd:.6%}')

TONAレート(Aug29): 4.980583%


In [12]:
# 変動レグの将来価値とrate (テキストp88)
dlyCmp = (1+ dfCFdy.days[1:]/365 * dfCFdy.rate[1:]).cumprod()-1
print(f'将来価値: {dlyCmp.iloc[-1]*ntlAMT:,.4f}',
        f'ﾌｪｱﾚｰﾄ: {dlyCmp.iloc[-1]*365/7 :.6%}' )

将来価値: 9,559.6298 ﾌｪｱﾚｰﾄ: 4.984664%


In [13]:
# 補足： (cumprodをprodへ修正した簡便なコード)
dlyCmp = (1+ dfCFdy.days[1:]/365 * dfCFdy.rate[1:]).prod()-1
print(f'将来価値: {dlyCmp*ntlAMT:,.4f}',
        f'ﾌｪｱﾚｰﾄ: {dlyCmp*365/7 :.6%}' )

将来価値: 9,559.6298 ﾌｪｱﾚｰﾄ: 4.984664%


In [14]:
# cumprodとiloc[-1]
srEX1 = pd.Series([3,4,5]).cumprod() ; print('cumprod= \n',srEX1)
print('iloc[-1]=',srEX1.iloc[-1]) 

cumprod= 
 0     3
1    12
2    60
dtype: int64
iloc[-1]= 60


In [15]:
# 8月25日評価
tnFixDT = [jDT(2022,8,23), jDT(2022,8,24)]              # Tona fixing
tnFixRT = [     5.1/100,     5.2/100     ]
tonaIX.addFixings(tnFixDT, tnFixRT, True)
tradeDT = jDT(2022,8,25)  ; setEvDT(tradeDT)            # 日付修正

print('(8月25日評価)')                                  # スワップ評価
display( pd.DataFrame([  
    ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
    ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
    ['フェアスプレッド',swapOBJ.fairSpread()] ], columns=['計算結果', ''])\
                                                 .style.format({'':'{:.7f}'}) )
print('(変動レグ明細)')                                 # 変動レグ
display( mu.swapCashFlow(swapOBJ, tnCrvOBJ, 1).style.format(fmtSCF) )

print('(TONAフォワードレート明細)')                     # TONAフォワードレート
dfAug25 = mu.swapCashFlow(swapOBJdy, tnCrvOBJ, 1)
display( dfAug25.style.format(fmtSCF))
                                                        # 変動レグの将来価値とrate
dlyCmp = (1+ dfAug25.days[1:]/365 * dfAug25.rate[1:]).cumprod()-1
print(f'(hc)将来価値: {dlyCmp.iloc[-1]*ntlAMT:,.4f}',
        f'(hc)ﾌｪｱﾚｰﾄ: {dlyCmp.iloc[-1]*365/7 :.6%} ')

(8月25日評価)


,計算結果,
0,固定レグ時価,-9579.8771324
1,変動レグ時価,9647.8463823
2,スワップ時価 NPV,67.9692499
3,フェアレート,0.0503548
4,フェアスプレッド,-0.0003548


(変動レグ明細)


,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan%,nan,1.00000000
1,2022-08-29,2022-08-23,2022-08-30,2022-09-01,7,5.035475%,0.000%,"9,657.0754",0.99904433


(TONAフォワードレート明細)


,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan%,nan,1.00000000
1,2022-08-23,2022-08-23,2022-08-24,2022-08-24,1,5.100000%,0.000%,"1,397.2603",1.00000000
2,2022-08-24,2022-08-24,2022-08-25,2022-08-25,1,5.200000%,0.000%,"1,424.6575",1.00000000
3,2022-08-25,2022-08-25,2022-08-26,2022-08-26,1,4.991000%,0.000%,"1,367.3973",0.99986328
4,2022-08-26,2022-08-26,2022-08-29,2022-08-29,3,4.986708%,0.000%,"4,098.6642",0.99945364
5,2022-08-29,2022-08-29,2022-08-30,2022-08-30,1,4.984665%,0.000%,"1,365.6617",0.99931716


(hc)将来価値: 9,657.0754 (hc)ﾌｪｱﾚｰﾄ: 5.035475% 


**Sofr**

In [16]:
#from myABBR import * ; import myUtil as mu
crvDATA = [('depo','1d',5.31), ('swap','1m',5.32), ('swap','3m',5.38),
           ('swap','6m',5.46), ('swap','1y',5.45), ('swap','2y',5.01),
           ('swap','3y',4.67)]

In [17]:
def makeSofrCurve(crvDATA):
  
  # 1.指標金利オブジェクトと初期値設定
  sfCrvHDL = ql.RelinkableYieldTermStructureHandle()  
  sofrIX   = ql.Sofr(sfCrvHDL)
  # 2. HelperとSOFRカーブオブジェクト
  cHelper, sfParRT = [], []
  for knd, tnr, rt in crvDATA:
      if knd == 'depo':
          cHelper.append(ql.DepositRateHelper(mu.sqHDL(rt/100),sofrIX)) 
      if knd == 'swap':
          cHelper.append(ql.OISRateHelper(Tp2,pD(tnr),mu.sqHDL(rt/100),sofrIX))
      sfParRT.append(rt/100)                             # パーレート用リスト
  # カーブオブジェクト      
  sfCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp0, calUSs, cHelper, dcA360)
  sfCrvHDL.linkTo(sfCrvOBJ) ; sfCrvOBJ.enableExtrapolation()
  return [sofrIX, sfCrvOBJ, sfCrvHDL, sfParRT]      # 4つのオブジェクトを戻す

In [18]:
tradeDT = jDT(2023,9,26) ; setEvDT(tradeDT)
# SOFRカーブ作成
sofrIX, sfCrvOBJ, sfCrvHDL, sfParRT = makeSofrCurve(crvDATA)
dfSOFR  = pd.DataFrame([(dt.ISO(),df) for dt,df in sfCrvOBJ.nodes()],
                                                columns=['date','DF'])
print("決済日(reference): ", sfCrvOBJ.referenceDate().ISO())
dfSOFR.style.format(fmtSCF)

決済日(reference):  2023-09-26


,date,DF
0,2023-09-26,1.00000000
1,2023-09-27,0.99985252
2,2023-10-30,0.99499988
3,2023-12-28,0.98629210
4,2024-03-28,0.97285119
5,2024-09-30,0.94694952
6,2025-09-29,0.90534683
7,2026-09-28,0.87063975


In [19]:
settleDT = sofrIX.valueDate(tradeDT)
print('trade          : ', tradeDT.ISO())
print('settle         : ', settleDT.ISO())
print('--------------- ------------')
print('fixingDays     : ', sofrIX.fixingDays())
print('fixingDate     : ', sofrIX.fixingDate(settleDT).ISO(), '\n')
print('tenor          : ', sofrIX.tenor()) 
print('dayCounter     : ', sofrIX.dayCounter())
print('fixingCalendar : ', sofrIX.fixingCalendar())
print('maturityDate   : ', sofrIX.maturityDate(settleDT).ISO())

trade          :  2023-09-26
settle         :  2023-09-26
--------------- ------------
fixingDays     :  0
fixingDate     :  2023-09-26 

tenor          :  1D
dayCounter     :  Actual/360 day counter
fixingCalendar :  SOFR fixing calendar calendar
maturityDate   :  2023-09-27


In [20]:
# スワップ条件  payRcv:ql.OvernightIndexedSwap.Payer = 1
effDT,               matDT,    payRcv, payLag, ntlAMT,     cpnRT,    sprdRT  =\
jDT(2023,9,28), jDT(2025,9,28),  1,      2,   10_000_000,  5.0/100,   0.0

# スケジュール及びスワップオブジェクトの作成、エンジン設定
fixSCD  = ql.Schedule(effDT, matDT, pdFreqA, calUSf, mFLLW, mFLLW, dtGENb, EoMf)
swapOBJ = ql.OvernightIndexedSwap(
                      payRcv, ntlAMT, fixSCD, cpnRT, dcA360, sofrIX, sprdRT, payLag)
swapOBJ.setPricingEngine(ql.DiscountingSwapEngine(sfCrvHDL))
# 計算結果
pd.DataFrame([  
    ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
    ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
    ['フェアスプレッド',swapOBJ.fairSpread()] ], columns=['計算結果', ''])\
                                                 .style.format({'':'{:.5f}'})    

,計算結果,
0,固定レグ時価,-941481.78212
1,変動レグ時価,943363.37982
2,スワップ時価 NPV,1881.59771
3,フェアレート,0.05010
4,フェアスプレッド,-0.00010


In [21]:
# 休日リスト
print( calUSs.holidayList(jDT(2023,1,1), jDT(2023,12,31))[:6] )
print( calUSf.holidayList(jDT(2023,1,1), jDT(2023,12,31))[:6] )

(Date(2,1,2023), Date(16,1,2023), Date(20,2,2023), Date(7,4,2023), Date(29,5,2023), Date(19,6,2023))
(Date(2,1,2023), Date(16,1,2023), Date(20,2,2023), Date(29,5,2023), Date(19,6,2023), Date(4,7,2023))


In [22]:
# 変動レグ:1
dfFltCF = mu.swapCashFlow(swapOBJ, sfCrvOBJ, 1, dcA360)
dfFltCF.style.format(fmtSCF)

,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2023-09-28,nan,nan%,nan%,nan,0.99970512
1,2024-09-27,2023-09-28,2024-09-30,2024-10-02,368,5.525694%,0.000%,"557,111.1111",0.94671578
2,2025-09-26,2024-09-30,2025-09-29,2025-10-01,364,4.607845%,0.000%,"459,522.0608",0.90515240


In [23]:
# forward swap rate
effDF  = sfCrvOBJ.discount(iDT(dfFltCF.accEnd[1]))
matDF  = sfCrvOBJ.discount(iDT(dfFltCF.accEnd[2]))
annFCT = 364/360*matDF
print(f'forward swap rate: {(effDF-matDF)/annFCT:.6%}')

forward swap rate: 4.544724%


In [24]:
#from myABBR import *     ; import myUtil as mu
tradeDT = jDT(2023,9,26) ; setEvDT(tradeDT)
# 0.SOFRカーブ
crvDATA = [('depo','1d',5.31), ('swap','1m',5.32), ('swap','3m',5.38),
           ('swap','6m',5.46), ('swap','1y',5.45), ('swap','2y',5.01),
           ('swap','3y',4.67)]
sofrIX, sfCrvOBJ, sfCrvHDL, sfParRT = mu.makeSofrCurve(crvDATA)

# 1.CME Term SOFR rate and Basis curve ( = all zero)
TsfRT3m  = 5.38558
TsfCrvBS = [('6m',0.0), ('1y',0.0), ('2y',0.0), ('3y',0.0) ]

# 2.TermSOFR指標金利オブジェクト
TsfCrvHDL= ql.RelinkableYieldTermStructureHandle()  
TsfIX    = ql.IborIndex('TermSofr', pdFreqQ, Tp2, usdFX, calUSs, mFLLW,
                                                          EoMt, dcA360, TsfCrvHDL)
# 3. Basis helperでのTermSOFRカーブオブジェクト
cHelper  = [ql.DepositRateHelper(mu.sqHDL(TsfRT3m/100),TsfIX)]
for tnr, bs in TsfCrvBS:
    cHelper.append(ql.OvernightIborBasisSwapRateHelper(mu.sqHDL(bs/100), 
                      pD(tnr), Tp2, calUSs, mFLLW, EoMf, sofrIX, TsfIX, sfCrvHDL))
TsfCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calUSs, cHelper, dcA360)
TsfCrvHDL.linkTo(TsfCrvOBJ) ; TsfCrvOBJ.enableExtrapolation()
# checking
print('決済日(reference):', TsfCrvOBJ.referenceDate().ISO())
[(dt.ISO(),df) for dt,df in TsfCrvOBJ.nodes()]

決済日(reference): 2023-09-28


[('2023-09-28', 1.0),
 ('2023-12-28', 0.9865692901876598),
 ('2024-03-28', 0.9731383278012513),
 ('2024-09-30', 0.947229008881084),
 ('2025-09-29', 0.9056140456298556),
 ('2026-09-28', 0.8708967171925004)]

**Two curve**

In [25]:
setEvDT(jDT(2021,8,2)) #from myABBR import * ; import myUtil as mu 

# Forward curve 金利
sCrvHDL = ql.RelinkableYieldTermStructureHandle()  
Rt1y,    Rt2y,       IX1y                                         =\
0.07,    0.13,   ql.Tibor(pdFreqA, sCrvHDL)
# カーブ構築
sHelper = [ql.DepositRateHelper(Rt1y, IX1y), 
           ql.SwapRateHelper(Rt2y,pD('2y'),calJP,freqA,mFLLW,dcA365,IX1y)]
sCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, sHelper, dcA365)
sCrvHDL.linkTo(sCrvOBJ)
# DF表示
dfSG = pd.DataFrame([(dt.ISO(),df,sCrvOBJ.zeroRate(dt,dcA365,cmpdSPL,freqA).rate())
                      for dt,df in sCrvOBJ.nodes()],columns=['date','DF','zeroRT']) 
display(dfSG.style.format(fmtSCF))
# DF手計算
print(f'1yr DF(hc): {1/(1+Rt1y)                   :.8f}, ',     # 1yr DF
      f'2yr DF(hc): {(1-Rt2y*dfSG.DF[1])/(1+Rt2y) :.8f}' )      # 2yr DF
# 1x2 fwd IX計算
fwdIX_1x2 = sCrvOBJ.forwardRate(iDT(dfSG.date[1]),iDT(dfSG.date[2]),dcA365,cmpdSPL)
print(f'1x2 fwdIX:{fwdIX_1x2.rate()        :.4%}, ', 
           f'(hc):{dfSG.DF[1]/dfSG.DF[2]-1 :.4%}'   )

,date,DF,zeroRT
0,2021-08-04,1.00000000,6.765888%
1,2022-08-04,0.93457944,7.000000%
2,2023-08-04,0.77743776,14.313830%


1yr DF(hc): 0.93457944,  2yr DF(hc): 0.77743776
1x2 fwdIX:20.2128%,  (hc):20.2128%


In [26]:
# スワップOBJ
swapTNR,   cpnRT,   fwdStart                                        =\
pD('2y'),  Rt2y,    pD('0d')
swapOBJ = ql.MakeVanillaSwap(swapTNR, IX1y, cpnRT, fwdStart, 
                            fixedLegTenor=pdFreqA, nominal=10_000_000)
# シングルカーブ評価
swapOBJ.setPricingEngine(ql.DiscountingSwapEngine(sCrvHDL))
print(f'NPV(single curve):{swapOBJ.NPV():.6f}')
display(mu.swapCashFlow(swapOBJ, sCrvOBJ).style.format(fmtSCF))

NPV(single curve):0.000000


,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2021-08-04,nan,nan%,nan%,nan,1.00000000
1,2021-08-02,2021-08-04,2022-08-04,2022-08-04,365,7.000000%,0.000%,"700,000.0000",0.93457944
2,2022-08-02,2022-08-04,2023-08-04,2023-08-04,365,20.212766%,0.000%,"2,021,276.5957",0.77743776


In [27]:
# Discount curve 金利
dCrvHDL = ql.RelinkableYieldTermStructureHandle()
dRt1y,  dRt2y,     dIX1y                          =\
0.05,   0.08,   ql.Tibor(pdFreqA, dCrvHDL)
# カーブ構築  
dHelper = [ql.DepositRateHelper(dRt1y, dIX1y), 
            ql.SwapRateHelper(dRt2y,pD('2y'),calJP, freqA, mFLLW, dcA365, dIX1y)]
dCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, dHelper, dcA365)
dCrvHDL.linkTo(dCrvOBJ)
# DF表示
dfDS = pd.DataFrame([(dt.ISO(),df) for dt,df in dCrvOBJ.nodes()],columns=['date','DF'])
display(dfDS.style.format({'DF':'{:.8f}'}))

,date,DF
0,2021-08-04,1.00000000
1,2022-08-04,0.95238095
2,2023-08-04,0.85537919


In [28]:
# forward curve
fCrvHDL = ql.RelinkableYieldTermStructureHandle()  
fIX1y   = ql.Tibor(pdFreqA, fCrvHDL)
# カーブ構築 
fHelper = [ql.DepositRateHelper(Rt1y, fIX1y),
            ql.SwapRateHelper(Rt2y, pD('2y'), calJP, freqA,
                 mFLLW, dcA365, fIX1y, mu.sqHDL(0), pD('0d'), dCrvHDL)]
fCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, fHelper, dcA365)
fCrvHDL.linkTo(fCrvOBJ)
# DF, fwdRT表示
dfFW    = pd.DataFrame(
    [(dt.ISO(),df) for dt,df in fCrvOBJ.nodes()],columns=['date','DF'])
display(dfFW.style.format({'DF':'{:.8f}'}))
fwdIX_1x2= fCrvOBJ.forwardRate(iDT(dfFW.date[1]),iDT(dfFW.date[2]),dcA365,cmpdSPL)
print(f'1x2 fwdIX:{fwdIX_1x2.rate():.4%}')

,date,DF
0,2021-08-04,1.00000000
1,2022-08-04,0.93457944
2,2023-08-04,0.78089590


1x2 fwdIX:19.6804%


* フォワードカーブのDFから、forward rateの計算は可能
* フォワードカーブのスワップレートは ディスカウントカーブのDFから算出する

In [29]:
t1 = ( iDT(dfFW.date[1]) - iDT(dfFW.date[0]) )/365
t2 = ( iDT(dfFW.date[2]) - iDT(dfFW.date[1]) )/365
hcFwdIX = (dfFW.DF[1]/dfFW.DF[2]-1)/t2

leg1 = 0.07*t1*dfDS.DF[1]+fwdIX_1x2.rate()*t2*dfDS.DF[2] 
ann  = t1*dfDS.DF[1]+t2*dfDS.DF[2] ; swp2y = leg1/ann 
print(f'(hc 1x2 fwd) {hcFwdIX:.4%}, ' f'(hc swap2Y) {swp2y:.4%}')

(hc 1x2 fwd) 19.6804%, (hc swap2Y) 13.0000%


In [30]:
# ベーシス調整DF
bs1y, bs2y = Rt1y-dRt1y, Rt2y-dRt2y              
a1DF = dfDS.DF[1] - bs1y* dfDS.DF[1]  
a2DF = dfDS.DF[2] - bs2y*(dfDS.DF[1]+dfDS.DF[2]) 
print(f'ベーシス調整DF 1y:{a1DF:.8f}, ', f'2y:{a2DF:.8f}')
# フォワードIbor, スワップレート
print(f'1x2 fwdIX(hc):{(a1DF - a2DF)/dfDS.DF[2]          :.4%}, ',   
        f'2y swap(hc):{(1 - a2DF)/(dfDS.DF[1]+dfDS.DF[2]):.4%}'  )

ベーシス調整DF 1y:0.93333333,  2y:0.76499118
1x2 fwdIX(hc):19.6804%,  2y swap(hc):13.0000%


In [31]:
# fIX1y指数のスワップOBJとエンジンの設定
swapOBJ2 = ql.MakeVanillaSwap(swapTNR, fIX1y, cpnRT, fwdStart, 
                                      fixedLegTenor=pdFreqA, nominal=10_000_000)
swapOBJ2.setPricingEngine(ql.DiscountingSwapEngine(dCrvHDL))
# スワップ評価
print(f'NPV(Two curve):{swapOBJ2.NPV():,.2f}, '
      f'FixLeg:{swapOBJ2.legNPV(0)    :,.2f}, FltLeg:{swapOBJ2.legNPV(1):,.2f}')
# キャッシュフロー表
display(mu.swapCashFlow(swapOBJ2, fCrvOBJ, 1).style.format(fmtSCF))   # float
display(mu.swapCashFlow(swapOBJ2, dCrvOBJ, 0).style.format(fmtSCF))   # fix

NPV(Two curve):0.00, FixLeg:-2,350,088.18, FltLeg:2,350,088.18


,fixDate,accStart,accEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2021-08-04,nan,nan%,nan%,nan,1.00000000
1,2021-08-02,2021-08-04,2022-08-04,2022-08-04,365,7.000000%,0.000%,"700,000.0000",0.93457944
2,2022-08-02,2022-08-04,2023-08-04,2023-08-04,365,19.680412%,0.000%,"1,968,041.2371",0.78089590


,nominal,accStart,accEnd,payDate,days,rate,amount,DF
0,nan,nan,nan,2021-08-04,nan,nan%,nan,1.00000000
1,"10,000,000.00",2021-08-04,2022-08-04,2022-08-04,365,13.000000%,"1,300,000.0000",0.95238095
2,"10,000,000.00",2022-08-04,2023-08-04,2023-08-04,365,13.000000%,"1,300,000.0000",0.85537919
